<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW0402.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install deeppavlov

     |████████████████████████████████| 737kB 2.5MB/s 
     |████████████████████████████████| 2.8MB 15.3MB/s 
     |████████████████████████████████| 6.7MB 21.1MB/s 
     |████████████████████████████████| 10.1MB 35.1MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 2.1MB 45.7MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 25.2MB 1.4MB/s 
     |████████████████████████████████| 17.3MB 243kB/s 
     |████████████████████████████████| 163kB 48.0MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 61kB 140kB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 8.0MB 36.4MB/s 
     |████████████████████████████████| 317kB 43.3MB/s 
     |████████████████████████████████| 7.1MB 24.7MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 5.1MB 28.5MB/s 
    

In [0]:
!pip install tensorflow==1.15.0

In [0]:
!apt-get --yes install git

In [0]:
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [0]:
import pandas as pd

In [0]:
import csv

In [0]:
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/data.txt

In [0]:
f = open('data.txt', 'r', encoding='utf-8')
lines = f.readlines()
f.close

rows = []
for l in lines:
	l = [l.strip()]
	rows.append(l)


with open('data.csv', 'w', newline='') as csvf:
	writer = csv.writer(csvf)
	for r in rows:
		writer.writerow(r)

In [0]:
data = pd.read_csv('data.csv', header=None, names=['text'])

In [0]:
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/test.txt
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/train.txt
!wget https://raw.githubusercontent.com/Yan-Weijia/CompLing/master/HW04data/valid.txt

In [0]:
!rm data.txt

In [13]:
!ls *.txt

test.txt  train.txt  valid.txt


In [14]:
!head train.txt

20 B-DATE
октября I-DATE
1887 I-DATE
года I-DATE
в O
Петербурге B-PLACE
в O
Мариинском B-PLACE
театре I-PLACE
— O


In [0]:
import json
from deeppavlov import configs, build_model, train_model

In [16]:
with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as f:
    ner_config = json.load(f)

ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

ner_model = train_model(ner_config, download=True)

2020-01-23 06:19:16.999 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [03:54<00:00, 2.82MB/s]
2020-01-23 06:23:11.903 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2020-01-23 06:23:18.620 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...


[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
2020-01-23 06:23:23.808 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /content/tag.dict]


Using TensorFlow backend.





The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard 

...

{"valid": {"eval_examples_count": 1, "metrics": {"ner_f1": 55.4622, "ner_token_f1": 81.5166}, "time_spent": "0:00:04"}}


2020-01-23 06:28:30.860 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 394: processed 156 tokens with 36 phrases; found: 44 phrases; correct: 0.

precision:  50.00%; recall:  61.11%; FB1:  55.00

	DATE: precision:  87.50%; recall:  87.50%; F1:  87.50 8

	PERSON: precision:  22.22%; recall:  100.00%; F1:  36.36 18

	PLACE: precision:  46.15%; recall:  60.00%; F1:  52.17 13

	WORK_OF_ART: precision:  100.00%; recall:  35.71%; F1:  52.63 5




{"test": {"eval_examples_count": 1, "metrics": {"ner_f1": 55.0, "ner_token_f1": 74.1722}, "time_spent": "0:00:01"}}


2020-01-23 06:28:31.654 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/tag.dict]
2020-01-23 06:29:01.994 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/model]


INFO:tensorflow:Restoring parameters from /content/model


In [17]:
ner_model(['14 октября 1890 года', '1890—1917 годы'])

[[['14', 'октября', '1890', 'года'], ['1890', '—', '1917', 'годы']],
 [['B-DATE', 'I-DATE', 'I-DATE', 'I-DATE'],
  ['I-DATE', 'O', 'B-DATE', 'I-DATE']]]

In [18]:
ner_model(['Петр Чайковский', 'Чайковский', 'П. и. Чайковский'])

[[['Петр', 'Чайковский'], ['Чайковский'], ['П', '.', 'и', '.', 'Чайковский']],
 [['B-PERSON', 'B-PERSON'],
  ['B-PERSON'],
  ['I-PERSON', 'O', 'I-PERSON', 'O', 'B-PERSON']]]

In [19]:
ner_model(['опера «Евгений Онегин»', '«Воевода»'])

[[['опера', '«', 'Евгений', 'Онегин', '»'], ['«', 'Воевода', '»']],
 [['B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-PERSON', 'B-PERSON', 'I-WORK_OF_ART'],
  ['I-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART']]]

In [20]:
ner_model(['в Москве', 'Большой театр'])

[[['в', 'Москве'], ['Большой', 'театр']],
 [['O', 'B-PLACE'], ['B-PLACE', 'I-PLACE']]]

In [0]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))

In [22]:
marked

[[('1867', 'B-DATE'),
  ('год', 'I-DATE'),
  ('—', 'O'),
  ('исполнение', 'O'),
  ('Andante', 'B-PERSON'),
  ('и', 'O'),
  ('скерцо', 'B-PERSON'),
  ('из', 'O'),
  ('Первой', 'O'),
  ('симфонии', 'B-PERSON'),
  ('в', 'O'),
  ('Русском', 'B-PLACE'),
  ('музыкальном', 'B-PLACE'),
  ('обществе', 'I-PLACE'),
  ('в', 'O'),
  ('Санкт', 'B-PLACE'),
  ('-', 'O'),
  ('Петербурге', 'B-PLACE'),
  ('.', 'O')],
 [('1866', 'I-DATE'),
  ('—', 'O'),
  ('1867', 'B-DATE'),
  ('годы', 'I-DATE'),
  ('—', 'O'),
  ('написаны', 'O'),
  ('увертюра', 'O'),
  ('на', 'O'),
  ('датский', 'O'),
  ('гимн', 'B-PERSON'),
  ('и', 'O'),
  ('ряд', 'O'),
  ('фортепьянных', 'B-PERSON'),
  ('пьес', 'B-PERSON'),
  (':', 'O'),
  ('«', 'I-WORK_OF_ART'),
  ('Воспоминание', 'I-WORK_OF_ART'),
  ('о', 'I-WORK_OF_ART'),
  ('Гапсале', 'I-WORK_OF_ART'),
  ('»', 'I-WORK_OF_ART'),
  ('.', 'O')],
 [('1867', 'B-DATE'),
  ('год', 'I-DATE'),
  ('—', 'O'),
  ('начата', 'O'),
  ('работа', 'O'),
  ('над', 'O'),
  ('оперой', 'B-WORK_OF_ART'),